In [37]:
import imutils
from imutils import paths
import os
import cv2
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [38]:
data = []
labels = []
bboxs = []
imagePaths = []

In [39]:
for csvPath in paths.list_files('./dataset/annotations/', validExts=('.csv')):
    rows = open(csvPath).read().strip().split('\n')[1:]
    
    for row in rows:
        row = row.split(',')
        filename, _, _, label, startX, startY, endX, endY = row
        
        imagePath = os.path.sep.join(['dataset', 'image', label, filename])
        if os.path.exists(imagePath):
            image = cv2.imread(imagePath)
            h, w = image.shape[:2]
        
            startX = float(startX) / w
            startY = float(startY) / h
            endX = float(endX) / w
            endY = float(endY) / h
            
            image = load_img(imagePath, target_size=(150,150))
            image = img_to_array(image)
            
            data.append(image)
            labels.append(label)
            bboxs.append((startX, startY, endX, endY))
            imagePaths.append(imagePath)

In [40]:
labels

['Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Actinic Keratosis',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Basa Cell Carcinoma',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Eksim',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Flek Hitam',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes',
 'Herpes'

In [41]:
data = np.array(data, dtype='float32') / 255.
labels = np.array(labels)
bboxs = np.array(bboxs, dtype='float32')
imagePaths = np.array(imagePaths)

In [42]:
np.unique(labels)

array(['Actinic Keratosis', 'Basa Cell Carcinoma', 'Eksim', 'Flek Hitam',
       'Herpes', 'Kerutan', 'Milia', 'Rosacea', 'Vitiligo', 'jerawat'],
      dtype='<U19')

In [43]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)

In [44]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [45]:
labels = to_categorical(labels)
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [46]:
split = train_test_split(data, labels, bboxs, imagePaths, test_size=0.2, random_state=42)

In [47]:
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

In [48]:
dn121 = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_tensor=tf.keras.layers.Input(shape=(150,150,3)))
dn121.trainable = False

flatten = dn121.output
flatten = tf.keras.layers.BatchNormalization()(flatten)
flatten = tf.keras.layers.Flatten()(flatten)

In [49]:
bboxHead = tf.keras.layers.Dense(64, activation="relu")(flatten)
bboxHead = tf.keras.layers.Dense(32, activation="relu")(bboxHead)
bboxHead = tf.keras.layers.Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)

softmaxHead = tf.keras.layers.Dense(len(lb.classes_), activation="softmax", name="class_label")(flatten)

model = tf.keras.models.Model(inputs=dn121.input, outputs=(bboxHead, softmaxHead))

In [50]:
losses = {
    'class_label': 'categorical_crossentropy',
    'bounding_box': 'mean_squared_error'
}

lossWeights = {
    'class_label': 1.,
    'bounding_box': 1.
}

model.compile(loss=losses, optimizer=tf.keras.optimizers.Adam(0.00001), loss_weights=lossWeights, metrics=['accuracy'])
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_8 (ZeroPadding2  (None, 156, 156, 3)  0          ['input_6[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 75, 75, 64)   9408        ['zero_padding2d_8[0][0]']       
                                                                                            

 conv2_block4_1_conv (Conv2D)   (None, 38, 38, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 38, 38, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_2_conv (Conv2D)   (None, 38, 38, 32)   36864       ['conv2_block4_1_relu[0][0]']    
                                                                                                  
 conv2_blo

                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 19, 19, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 19, 19, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 19, 19, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 19, 19, 32)   36864       ['conv3_block2_1_relu[0][0]']    
          

 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 19, 19, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 19, 19, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_block7_0_bn (BatchNormal  (None, 19, 19, 320)  1280       ['conv3_block6_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_blo

 conv3_block11_0_bn (BatchNorma  (None, 19, 19, 448)  1792       ['conv3_block10_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_0_relu (Activati  (None, 19, 19, 448)  0          ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 19, 19, 128)  57344       ['conv3_block11_0_relu[0][0]']   
                                                                                                  
 conv3_block11_1_bn (BatchNorma  (None, 19, 19, 128)  512        ['conv3_block11_1_conv[0][0]']   
 lization)                                                                                        
          

...

                                                                                                  
 conv4_block24_concat (Concaten  (None, 9, 9, 1024)  0           ['conv4_block23_concat[0][0]',   
 ate)                                                             'conv4_block24_2_conv[0][0]']   
                                                                                                  
 pool4_bn (BatchNormalization)  (None, 9, 9, 1024)   4096        ['conv4_block24_concat[0][0]']   
                                                                                                  
 pool4_relu (Activation)        (None, 9, 9, 1024)   0           ['pool4_bn[0][0]']               
                                                                                                  
 pool4_conv (Conv2D)            (None, 9, 9, 512)    524288      ['pool4_relu[0][0]']             
                                                                                                  
 pool4_poo

 conv5_block4_2_conv (Conv2D)   (None, 4, 4, 32)     36864       ['conv5_block4_1_relu[0][0]']    
                                                                                                  
 conv5_block4_concat (Concatena  (None, 4, 4, 640)   0           ['conv5_block3_concat[0][0]',    
 te)                                                              'conv5_block4_2_conv[0][0]']    
                                                                                                  
 conv5_block5_0_bn (BatchNormal  (None, 4, 4, 640)   2560        ['conv5_block4_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block5_0_relu (Activatio  (None, 4, 4, 640)   0           ['conv5_block5_0_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv5_block9_0_relu (Activatio  (None, 4, 4, 768)   0           ['conv5_block9_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block9_1_conv (Conv2D)   (None, 4, 4, 128)    98304       ['conv5_block9_0_relu[0][0]']    
                                                                                                  
 conv5_block9_1_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv5_block9_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block9_1_relu (Activatio  (None, 4, 4, 128)   0           ['conv5_block9_1_bn[0][0]']      
 n)       

 lization)                                                                                        
                                                                                                  
 conv5_block13_1_relu (Activati  (None, 4, 4, 128)   0           ['conv5_block13_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block13_2_conv (Conv2D)  (None, 4, 4, 32)     36864       ['conv5_block13_1_relu[0][0]']   
                                                                                                  
 conv5_block13_concat (Concaten  (None, 4, 4, 928)   0           ['conv5_block12_concat[0][0]',   
 ate)                                                             'conv5_block13_2_conv[0][0]']   
                                                                                                  
 conv5_blo

                                                                                                  
Total params: 8,256,302
Trainable params: 1,216,750
Non-trainable params: 7,039,552
__________________________________________________________________________________________________
None


In [51]:
trainTargets = {
    'class_label': trainLabels,
    'bounding_box': trainBBoxes
}

testTargets = {
    'class_label': testLabels,
    'bounding_box': testBBoxes
}

In [53]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_class_label_accuracy') > 0.77):
      self.model.stop_training = True
cb = myCallback()

history = model.fit(trainImages, trainTargets, 
                    validation_data=(testImages, testTargets), 
                    batch_size=3, 
                    epochs=100,
                    verbose=1,
                    callbacks=cb)

Epoch 1/100
50/50 [==============================] - 12s 234ms/step - loss: 0.0297 - bounding_box_loss: 0.0140 - class_label_loss: 0.0157 - bounding_box_accuracy: 0.7450 - class_label_accuracy: 1.0000 - val_loss: 1.1206 - val_bounding_box_loss: 0.0401 - val_class_label_loss: 1.0805 - val_bounding_box_accuracy: 0.4474 - val_class_label_accuracy: 0.6842
Epoch 2/100
50/50 [==============================] - 12s 233ms/step - loss: 0.0285 - bounding_box_loss: 0.0133 - class_label_loss: 0.0152 - bounding_box_accuracy: 0.7584 - class_label_accuracy: 1.0000 - val_loss: 1.1234 - val_bounding_box_loss: 0.0399 - val_class_label_loss: 1.0834 - val_bounding_box_accuracy: 0.4474 - val_class_label_accuracy: 0.6842
Epoch 3/100
50/50 [==============================] - 12s 236ms/step - loss: 0.0279 - bounding_box_loss: 0.0137 - class_label_loss: 0.0143 - bounding_box_accuracy: 0.7450 - class_label_accuracy: 1.0000 - val_loss: 1.1212 - val_bounding_box_loss: 0.0396 - val_class_label_loss: 1.0817 - val_bou

50/50 [==============================] - 12s 246ms/step - loss: 0.0213 - bounding_box_loss: 0.0138 - class_label_loss: 0.0075 - bounding_box_accuracy: 0.7517 - class_label_accuracy: 1.0000 - val_loss: 1.0939 - val_bounding_box_loss: 0.0400 - val_class_label_loss: 1.0539 - val_bounding_box_accuracy: 0.4211 - val_class_label_accuracy: 0.6842
Epoch 25/100
50/50 [==============================] - 12s 236ms/step - loss: 0.0210 - bounding_box_loss: 0.0133 - class_label_loss: 0.0077 - bounding_box_accuracy: 0.7584 - class_label_accuracy: 1.0000 - val_loss: 1.0976 - val_bounding_box_loss: 0.0402 - val_class_label_loss: 1.0574 - val_bounding_box_accuracy: 0.4211 - val_class_label_accuracy: 0.6842
Epoch 26/100
50/50 [==============================] - 12s 234ms/step - loss: 0.0204 - bounding_box_loss: 0.0126 - class_label_loss: 0.0078 - bounding_box_accuracy: 0.7651 - class_label_accuracy: 1.0000 - val_loss: 1.0971 - val_bounding_box_loss: 0.0403 - val_class_label_loss: 1.0569 - val_bounding_box_

50/50 [==============================] - 14s 282ms/step - loss: 0.0182 - bounding_box_loss: 0.0132 - class_label_loss: 0.0050 - bounding_box_accuracy: 0.7919 - class_label_accuracy: 1.0000 - val_loss: 1.0936 - val_bounding_box_loss: 0.0400 - val_class_label_loss: 1.0536 - val_bounding_box_accuracy: 0.4211 - val_class_label_accuracy: 0.7105
Epoch 48/100
50/50 [==============================] - 14s 289ms/step - loss: 0.0211 - bounding_box_loss: 0.0135 - class_label_loss: 0.0076 - bounding_box_accuracy: 0.7785 - class_label_accuracy: 1.0000 - val_loss: 1.0938 - val_bounding_box_loss: 0.0398 - val_class_label_loss: 1.0540 - val_bounding_box_accuracy: 0.4211 - val_class_label_accuracy: 0.7105
Epoch 49/100
50/50 [==============================] - 15s 306ms/step - loss: 0.0186 - bounding_box_loss: 0.0129 - class_label_loss: 0.0057 - bounding_box_accuracy: 0.7852 - class_label_accuracy: 1.0000 - val_loss: 1.0957 - val_bounding_box_loss: 0.0403 - val_class_label_loss: 1.0553 - val_bounding_box_

50/50 [==============================] - 11s 218ms/step - loss: 0.0157 - bounding_box_loss: 0.0129 - class_label_loss: 0.0028 - bounding_box_accuracy: 0.7919 - class_label_accuracy: 1.0000 - val_loss: 1.0962 - val_bounding_box_loss: 0.0399 - val_class_label_loss: 1.0563 - val_bounding_box_accuracy: 0.4474 - val_class_label_accuracy: 0.6842
Epoch 71/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0157 - bounding_box_loss: 0.0128 - class_label_loss: 0.0029 - bounding_box_accuracy: 0.7785 - class_label_accuracy: 1.0000 - val_loss: 1.0997 - val_bounding_box_loss: 0.0398 - val_class_label_loss: 1.0599 - val_bounding_box_accuracy: 0.4474 - val_class_label_accuracy: 0.6842
Epoch 72/100
50/50 [==============================] - 11s 216ms/step - loss: 0.0157 - bounding_box_loss: 0.0132 - class_label_loss: 0.0025 - bounding_box_accuracy: 0.7852 - class_label_accuracy: 1.0000 - val_loss: 1.0980 - val_bounding_box_loss: 0.0398 - val_class_label_loss: 1.0581 - val_bounding_box_

50/50 [==============================] - 11s 216ms/step - loss: 0.0143 - bounding_box_loss: 0.0126 - class_label_loss: 0.0018 - bounding_box_accuracy: 0.8054 - class_label_accuracy: 1.0000 - val_loss: 1.0957 - val_bounding_box_loss: 0.0390 - val_class_label_loss: 1.0567 - val_bounding_box_accuracy: 0.4737 - val_class_label_accuracy: 0.6842
Epoch 94/100
50/50 [==============================] - 11s 212ms/step - loss: 0.0140 - bounding_box_loss: 0.0127 - class_label_loss: 0.0013 - bounding_box_accuracy: 0.7785 - class_label_accuracy: 1.0000 - val_loss: 1.0976 - val_bounding_box_loss: 0.0396 - val_class_label_loss: 1.0580 - val_bounding_box_accuracy: 0.4737 - val_class_label_accuracy: 0.6842
Epoch 95/100
50/50 [==============================] - 11s 217ms/step - loss: 0.0134 - bounding_box_loss: 0.0120 - class_label_loss: 0.0014 - bounding_box_accuracy: 0.7584 - class_label_accuracy: 1.0000 - val_loss: 1.0920 - val_bounding_box_loss: 0.0395 - val_class_label_loss: 1.0525 - val_bounding_box_

In [54]:
model.evaluate(testImages)

2/2 [==============================] - 2s 11ms/step - loss: 0.0000e+00 - bounding_box_loss: 0.0000e+00 - class_label_loss: 0.0000e+00 - bounding_box_accuracy: 0.0000e+00 - class_label_accuracy: 0.0000e+00


[0.0, 0.0, 0.0, 0.0, 0.0]

In [56]:
for n in range(10,20):
    image = load_img(imagePaths[n], target_size=(150,150))
    image = img_to_array(image) / 255.
    image = np.expand_dims(image, axis=0)

    box_preds, label_preds = model.predict(image)
    startX, startY, endX, endY = box_preds[0]
    i = np.argmax(label_preds, axis=1)
    label = lb.classes_[i][0]

    image = cv2.imread(imagePaths[n])
    image = imutils.resize(image, width=600)
    h, w = image.shape[:2]

    startX = int(startX * w) 
    startY = int(startY * h)
    endX = int(endX * w)
    endY = int(endY * h)

    teks = label + ' : ' + str(np.round(label_preds[0][i][0] * 100, 2)) + '%'
    print(teks)
    #print(startX, startY, endX, endY)

    y = startY - 20 if startY - 20 > 20 else startY + 20
    cv2.putText(image, teks, (startX + 5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.rectangle(image, (startX, startY), (endX, endY), (0,255,0), 2)
    cv2.imshow('output', image)
    cv2.waitKey(0)

1/1 [==============================] - 0s 91ms/step
Basa Cell Carcinoma : 99.92%
1/1 [==============================] - 0s 113ms/step
Basa Cell Carcinoma : 100.0%
1/1 [==============================] - 0s 110ms/step
Basa Cell Carcinoma : 99.95%
1/1 [==============================] - 0s 108ms/step
Basa Cell Carcinoma : 100.0%
1/1 [==============================] - 0s 110ms/step
Basa Cell Carcinoma : 99.86%
1/1 [==============================] - 0s 110ms/step
Kerutan : 88.21%
1/1 [==============================] - 0s 100ms/step
Basa Cell Carcinoma : 60.84%
1/1 [==============================] - 0s 110ms/step
Basa Cell Carcinoma : 99.96%
1/1 [==============================] - 0s 111ms/step
Basa Cell Carcinoma : 99.92%
1/1 [==============================] - 0s 110ms/step
Eksim : 83.48%


In [ ]:
model.save('./model/model75.h5')